In [1]:
import torch
import gpytorch
from botorch.models import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.transforms.outcome import Standardize
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_torch # fix Cholecky jitter error
from scipy.stats import norm
from sklearn.decomposition import PCA
import math 
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
import os
import time

# config plot settings
plt.rcParams["font.size"] = 16

In [2]:
###
#  Load Data
###
file = h5py.File("targets_and_normalized_features.jld2", "r")
# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data (high fidelity)
y = torch.from_numpy(np.transpose(file["gcmc_y"][:]))
# associated simulation costs
cost = torch.from_numpy(np.transpose(file["gcmc_elapsed_time"][:]))

# total number of COFs in data set
nb_COFs = X.shape[0]

print("raw data - \n\tX:", X.shape)
print("\t\ty:", y.shape)
print("\t\tcost: ", cost.shape)    
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
		y: torch.Size([608])
		cost:  torch.Size([608])

Ensure features are normalized - 
max:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
min:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)


In [3]:
X_unsqueezed = X.unsqueeze(1)

## Helper Functions

#### Construct Initial Inputs

In [4]:
# find COF closest to the center of feature space
def get_initializing_COF(X):
    # center of feature space
    feature_center = np.ones(X.shape[1]) * 0.5
    # max possible distance between normalized features
    return np.argmin(np.linalg.norm(X - feature_center, axis=1))

# yields np.array([25, 494, 523])
def diverse_set(X, train_size):
    # initialize with one random point; pick others in a max diverse fashion
    ids_train = [get_initializing_COF(X)]
    # select remaining training points
    for j in range(train_size - 1):
        # for each point in data set, compute its min dist to training set
        dist_to_train_set = np.linalg.norm(X - X[ids_train, None, :], axis=2)
        assert np.shape(dist_to_train_set) == (len(ids_train), nb_COFs)
        min_dist_to_a_training_pt = np.min(dist_to_train_set, axis=0)
        assert np.size(min_dist_to_a_training_pt) == nb_COFs
        
        # acquire point with max(min distance to train set) i.e. Furthest from train set
        ids_train.append(np.argmax(min_dist_to_a_training_pt))
    assert np.size(np.unique(ids_train)) == train_size # must be unique
    return np.array(ids_train)

In [5]:
# construct feature matrix of acquired points
def build_X_train(ids_acquired):
    return X[ids_acquired, :]

# construct output vector for acquired points
def build_y_train(ids_acquired):
    return y[ids_acquired].unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(ids_acquired):
    return cost[ids_acquired]

In [6]:
# construct and fit GP model
def construct_and_fit_gp_model(X_train, y_train):      
    model = SingleTaskGP(X_train, y_train, outcome_transform=Standardize(m=1))
    mll   = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

#### Bayesian Algorithm

In [7]:
def run_Bayesian_optimization(nb_iterations, nb_COFs_initialization):
    assert nb_iterations > nb_COFs_initialization
    ###
    #  Initialize system
    ###
    # select initial COFs for training data
    ids_acquired = diverse_set(X, nb_COFs_initialization)
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(nb_COFs_initialization, nb_iterations):
        # construct training data (perform experiments)
        X_train = build_X_train(ids_acquired)
        y_train = build_y_train(ids_acquired)
        
        # fit GP surrogate model
        model = construct_and_fit_gp_model(X_train, y_train)
        
        # compute expected improvement acquisition function
        acq_fn   = ExpectedImprovement(model, best_f=y_train.max().item())
        acq_vals = acq_fn.forward(X.unsqueeze(1))
        
        # identify COF with highest acquisition value currently *not* in training set
        ids_sorted_by_aquisition = acq_vals.argsort(descending=True)
        for id_max in ids_sorted_by_aquisition:
            if not id_max.item() in ids_acquired:
                id_max_acq = id_max.item()
                break
                
        ###
        #  acquire this COF
        ###
        ids_acquired = np.concatenate((ids_acquired, [id_max_acq]))
        print("BO iteration ", i)
        print("\tacquired COF ", id_max_acq)
        print("\ty = ", y[id_max_acq].item())
    
    # check budget constraint is stisfied
    assert np.size(ids_acquired) == nb_iterations
    assert len(np.unique(ids_acquired)) == nb_iterations
    return ids_acquired

# Run BO

In [15]:
ids_acquired = diverse_set(X, 3)
ids_acquired

array([ 25,   9, 494])

In [8]:
nb_iterations = 150
nb_COFs_initialization = 3

ids_acquired = run_Bayesian_optimization(nb_iterations, nb_COFs_initialization)

y_acquired    = y[ids_acquired]
cost_acquired = build_cost(ids_acquired)

Iter 10/100: 2.5319261569948535
Iter 20/100: 1.5977213829052925
Iter 30/100: 1.533526788169449
Iter 40/100: 1.510463070658428
Iter 50/100: 1.467186006191114
Iter 60/100: 1.4641999759046165
BO iteration  3
	acquired COF  6
	y =  2.608942041674406
Iter 10/100: 2.300540755501195
Iter 20/100: 1.5942759316124775
Iter 30/100: 1.5381509127916257
Iter 40/100: 1.51412358383249
Iter 50/100: 1.482890838679763
Iter 60/100: 1.4812470406796467
BO iteration  4
	acquired COF  35
	y =  3.5779583041972387
Iter 10/100: 2.151953314118534
Iter 20/100: 1.5806670209769442
Iter 30/100: 1.523087630942318
Iter 40/100: 1.4911371033400465
Iter 50/100: 1.4663851356157576
Iter 60/100: 1.465069457451125
Iter 70/100: 1.4641753882855182
Iter 80/100: 1.4626038615592214
BO iteration  5
	acquired COF  79
	y =  9.279758219433191
Iter 10/100: 2.0537156529756593
Iter 20/100: 1.570151540814714
Iter 30/100: 1.507336163946085
Iter 40/100: 1.4740459693527477
Iter 50/100: 1.4508893033755752
Iter 60/100: 1.4484263963564157
BO ite

Iter 20/100: 1.4420604066656353
Iter 30/100: 1.3730377635995357
Iter 40/100: 1.262368210137755
BO iteration  38
	acquired COF  523
	y =  14.017515356923804
Iter 10/100: 1.577031554283469
Iter 20/100: 1.4380000278729885
Iter 30/100: 1.362355034190661
Iter 40/100: 1.22303682690739
BO iteration  39
	acquired COF  524
	y =  6.895659242359536
Iter 10/100: 1.5752571742044919
Iter 20/100: 1.4378351376357856
Iter 30/100: 1.3642821036622457
Iter 40/100: 1.233741618469238
BO iteration  40
	acquired COF  45
	y =  6.90575169509638
Iter 10/100: 1.5736239680339752
Iter 20/100: 1.437454384339105
Iter 30/100: 1.3661835152921318
Iter 40/100: 1.2452586435801347
BO iteration  41
	acquired COF  73
	y =  4.268672491379296
Iter 10/100: 1.5713732253931
Iter 20/100: 1.4361397872701036
Iter 30/100: 1.3654981590775834
Iter 40/100: 1.2476722976377135
BO iteration  42
	acquired COF  302
	y =  2.800063705873911
Iter 10/100: 1.5678332584577057
Iter 20/100: 1.433778705386645
Iter 30/100: 1.3622591608882402
Iter 40/1

Iter 100/100: 1.1331140324038798
BO iteration  80
	acquired COF  492
	y =  2.912056948895514
Iter 10/100: 1.5059859771269621
Iter 20/100: 1.393336771701883
Iter 30/100: 1.311299015739467
Iter 40/100: 1.2278058554832278
Iter 50/100: 1.1520423489070315
Iter 60/100: 1.140453850901125
Iter 70/100: 1.139161090892152
Iter 80/100: 1.1374582058537985
Iter 90/100: 1.1371096647586458
Iter 100/100: 1.1369865741720304
BO iteration  81
	acquired COF  41
	y =  6.231560962484038
Iter 10/100: 1.5051038734739448
Iter 20/100: 1.3928345188443114
Iter 30/100: 1.3103313403530839
Iter 40/100: 1.2256140043768629
Iter 50/100: 1.1496059301016244
Iter 60/100: 1.1378949767706021
Iter 70/100: 1.1365644947500408
Iter 80/100: 1.134947309266919
Iter 90/100: 1.1345654390654378
Iter 100/100: 1.1344665933494893
BO iteration  82
	acquired COF  370
	y =  3.8803155027900575
Iter 10/100: 1.5040459795770584
Iter 20/100: 1.3918862038538589
Iter 30/100: 1.3090494081570703
Iter 40/100: 1.2244236774626145
Iter 50/100: 1.1488438

Iter 100/100: 1.1018436786044874
BO iteration  102
	acquired COF  199
	y =  3.537354772562271
Iter 10/100: 1.4834511952764977
Iter 20/100: 1.3726236507796539
Iter 30/100: 1.2757505593727494
Iter 40/100: 1.1808943732461292
Iter 50/100: 1.1131982666304159
Iter 60/100: 1.101619761078945
Iter 70/100: 1.1010043378333194
Iter 80/100: 1.0992300166230602
Iter 90/100: 1.0990119443900803
Iter 100/100: 1.098857585086588
BO iteration  103
	acquired COF  136
	y =  3.47101430668614
Iter 10/100: 1.4824436281520121
Iter 20/100: 1.3715691199678182
Iter 30/100: 1.2738933277495705
Iter 40/100: 1.1780758316634043
Iter 50/100: 1.1107712011478648
Iter 60/100: 1.099076874190053
Iter 70/100: 1.0984706968395612
Iter 80/100: 1.0966873938100778
Iter 90/100: 1.0964676162544995
Iter 100/100: 1.0963147539460723
BO iteration  104
	acquired COF  367
	y =  2.1920438633713037
Iter 10/100: 1.480982630548437
Iter 20/100: 1.3697759877997011
Iter 30/100: 1.270559003452879
Iter 40/100: 1.1730968916794704
Iter 50/100: 1.1062

Iter 70/100: 1.1024381328563098
Iter 80/100: 1.1013126978731642
Iter 90/100: 1.100907669166611
BO iteration  126
	acquired COF  310
	y =  3.7485008478321493
Iter 10/100: 1.463588032330788
Iter 20/100: 1.3509080087541376
Iter 30/100: 1.2405243342657908
Iter 40/100: 1.1382813783884709
Iter 50/100: 1.1078461062714418
Iter 60/100: 1.1015075722941952
Iter 70/100: 1.0999549799503145
Iter 80/100: 1.0988352127325602
Iter 90/100: 1.0984190574071317
BO iteration  127
	acquired COF  1
	y =  3.272498162401613
Iter 10/100: 1.4627572448564983
Iter 20/100: 1.3499987430959848
Iter 30/100: 1.2389583237239359
Iter 40/100: 1.135898109828286
Iter 50/100: 1.1051568538067944
Iter 60/100: 1.0988810161430809
Iter 70/100: 1.0972606667702918
Iter 80/100: 1.0961464789103865
Iter 90/100: 1.095727835130705
BO iteration  128
	acquired COF  217
	y =  3.9346540121679805
Iter 10/100: 1.4627491205587786
Iter 20/100: 1.350343512096523
Iter 30/100: 1.239767002778986
Iter 40/100: 1.137023495919859
Iter 50/100: 1.105698641

In [9]:
# accounts for ratification step i.e. the extra step need to perform experiment and verify the property 
BO_iter_top_cof_acquired = np.argmax(y_acquired == max(y))
top_cof_acc_cost = sum(cost_acquired[:BO_iter_top_cof_acquired]).item() 

print("iteration we acquire top COF = ", BO_iter_top_cof_acquired.item())
print("accumulated cost up to observation of top COF = ", top_cof_acc_cost, " [min]")

iteration we acquire top COF =  26
accumulated cost up to observation of top COF =  9731.945460963248  [min]


In [10]:
###
#  Store SFBO results
###
sfbo_res = dict({'ids_acquired': ids_acquired,
                 'y_acquired': y_acquired.detach().numpy(),
                 'cost_acquired': cost_acquired.detach().numpy(),
                 'nb_COFs_initialization': nb_COFs_initialization,
                 'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired.item()
                })

with open('search_results/sfbo_results_with_EI.pkl', 'wb') as file:
    pickle.dump(sfbo_res, file)

# Random Search

In [11]:
nb_runs = 1000

rs_res = dict()
rs_res['ids_acquired'] = []
rs_res['cost_acquired'] = []

for r in range(nb_runs):
    rs_ids_acquired = np.random.choice(range(nb_COFs), replace=False, size=nb_iterations)
    rs_cost_acquired = build_cost(rs_ids_acquired)
    rs_res['ids_acquired'].append(rs_ids_acquired)
    rs_res['cost_acquired'].append(rs_cost_acquired)

In [12]:
def test_rs(rs_res):
    for a in np.random.choice(nb_runs, replace=False, size=7):
        # check that the lengths 
        assert len(rs_res['ids_acquired'][a]) == nb_iterations
        assert all([cof_id in range(nb_COFs) for cof_id in rs_res['ids_acquired'][a]])
    return

test_rs(rs_res)

In [13]:
# get y_max acquired up to iteration i for i = 1,2,...
def y_max(rs_ids_acquired):
    y_max_mu      = np.zeros(nb_iterations)
    y_max_sig_bot = np.zeros(nb_iterations)
    y_max_sig_top = np.zeros(nb_iterations)
    # element i of these will be y max at BO iter i
    
    for i in range(1, nb_iterations+1):
        # max value acquired up to this point
        y_maxes = np.array([max(y[rs_ids_acquired[r][:i]]) for r in range(nb_runs)])
        assert np.size(y_maxes) == nb_runs
        y_max_mu[i-1]      = np.mean(y_maxes)
        y_max_sig_bot[i-1] = np.std(y_maxes[y_maxes < y_max_mu[i-1]])
        y_max_sig_top[i-1] = np.std(y_maxes[y_maxes > y_max_mu[i-1]])
    return y_max_mu, y_max_sig_bot, y_max_sig_top

# rs_mean, rs_lower_bound, rs_upper_bound = y_max(rs_res)
y_rs_max_mu, y_rs_max_sig_bot, y_rs_max_sig_top = y_max(rs_res['ids_acquired'])

In [14]:
###
#  Store Random Search Results
###
random_search_results = dict({'ids_acquired': rs_res['ids_acquired'],
                             'y_rs_max_mu': y_rs_max_mu,
                             'y_rs_max_sig_bot': y_rs_max_sig_bot,
                             'y_rs_max_sig_top': y_rs_max_sig_top,
                             'cost_acquired': rs_res['cost_acquired']
                             })

with open('search_results/random_search_results.pkl', 'wb') as file:
    pickle.dump(random_search_results, file)